<a href="https://colab.research.google.com/github/andervies/divic-corp-machine-learning-course/blob/main/assignment28/Keras_Series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Problem One: Sharing and executing the official tutorial model

Import TensorFlow to get started

In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.15.0


Load and prepare the MNIST dataset. The pixel values of the images range from 0 through 255. Scale these values to a range of 0 to 1 by dividing the values by 255.0. This also converts the sample data from integers to floating-point numbers:

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 [==============================] - 0s 0us/step


Build a tf.keras.Sequential model

In [5]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

Making predictions using the untrained model

In [6]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.09701695, -0.42782122,  0.12474165,  0.4498067 , -0.7428633 ,
         0.23040389,  0.48080975, -0.23871507,  0.49615145, -0.4797041 ]],
      dtype=float32)

The tf.nn.softmax function converts these logits to probabilities for each class

In [7]:
tf.nn.softmax(predictions).numpy()

array([[0.10150041, 0.06005283, 0.10435385, 0.14443833, 0.04382404,
        0.11598371, 0.14898649, 0.07255398, 0.15128982, 0.05701656]],
      dtype=float32)

Define a loss function for training using `losses.SparseCategoricalCrossentropy`:

In [8]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to -tf.math.log(1/10) ~= 2.3

In [9]:
loss_fn(y_train[:1], predictions).numpy()

2.1543055

Configuring and compiling the model using Keras Model.compile before beginning training

In [10]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

Use the Model.fit method to adjust model parameters and minimize the loss

In [11]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 11s 5ms/step - loss: 0.2935 - accuracy: 0.9147
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1436 - accuracy: 0.9571
Epoch 3/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1075 - accuracy: 0.9674
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0866 - accuracy: 0.9732
Epoch 5/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0762 - accuracy: 0.9760


The Model.evaluate method checks the model's performance

In [12]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0729 - accuracy: 0.9792 - 687ms/epoch - 2ms/step


[0.07294416427612305, 0.979200005531311]

To get our model to return a probability, we can wrap the trained model in the Sequential class, and attach the softmax to it

In [13]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [14]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.6523332e-08, 1.4008761e-08, 4.3974401e-06, 5.8098417e-04,
        6.9921437e-11, 2.4120897e-07, 4.5561482e-13, 9.9941361e-01,
        2.1695135e-07, 5.5293322e-07],
       [4.1952379e-07, 2.0351608e-05, 9.9994695e-01, 2.9258730e-05,
        4.6562450e-13, 9.5160954e-07, 3.2780616e-08, 1.8639070e-12,
        2.0479163e-06, 1.0462100e-11],
       [6.9884862e-07, 9.9767119e-01, 4.2209239e-04, 5.8585880e-05,
        3.8834587e-05, 4.1444027e-06, 9.8606233e-06, 1.2903715e-03,
        5.0147570e-04, 2.7855110e-06],
       [9.9992609e-01, 4.8043449e-09, 5.8983205e-05, 6.5251626e-08,
        5.1328268e-08, 1.6541933e-06, 1.1312318e-05, 1.4476245e-06,
        1.6638952e-08, 3.6356832e-07],
       [6.4561045e-06, 2.0696733e-08, 1.9558477e-06, 1.1374278e-08,
        9.9749005e-01, 7.6075928e-08, 1.8874531e-05, 1.1320080e-03,
        4.5496768e-06, 1.3459733e-03]], dtype=float32)>

The model trains to ~98% accuracy on this dataset and by applying the softmax function on the outputs of the trained model, we can make probabilistic classifications.

## Problem Three: Learning Iris (binary classification) with Keras

In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.datasets import mnist
from keras.optimizers import Adam

In [16]:
# Load dataset
iris_df = pd.read_csv("/content/drive/MyDrive/assignment28/Iris.csv")

# Filter only versicolor and virginica species
iris_df = iris_df[(iris_df["Species"] == "Iris-versicolor") | (iris_df["Species"] == "Iris-virginica")]

# Extract features and labels
X = iris_df[["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]].values
y = iris_df["Species"].replace({"Iris-versicolor": 0, "Iris-virginica": 1}).astype(np.int64).values

# Split into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
X_test, y_test = X_temp, y_temp

# Hyperparameters
learning_rate = 0.001
batch_size = 10
num_epochs = 100

# Neural network model
model = Sequential([
    Dense(50, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(100, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(X_val, y_val), verbose=1)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_acc)

Epoch 1/100
7/7 [==============================] - 1s 40ms/step - loss: 0.6820 - accuracy: 0.5312 - val_loss: 0.6831 - val_accuracy: 0.4375
Epoch 2/100
7/7 [==============================] - 0s 10ms/step - loss: 0.6496 - accuracy: 0.5938 - val_loss: 0.6666 - val_accuracy: 0.4375
Epoch 3/100
7/7 [==============================] - 0s 8ms/step - loss: 0.6288 - accuracy: 0.5469 - val_loss: 0.6634 - val_accuracy: 0.4375
Epoch 4/100
7/7 [==============================] - 0s 7ms/step - loss: 0.6098 - accuracy: 0.6406 - val_loss: 0.6229 - val_accuracy: 0.7500
Epoch 5/100
7/7 [==============================] - 0s 7ms/step - loss: 0.5858 - accuracy: 0.7812 - val_loss: 0.6131 - val_accuracy: 0.6875
Epoch 6/100
7/7 [==============================] - 0s 11ms/step - loss: 0.5723 - accuracy: 0.8125 - val_loss: 0.5898 - val_accuracy: 0.7500
Epoch 7/100
7/7 [==============================] - 0s 10ms/step - loss: 0.5505 - accuracy: 0.8281 - val_loss: 0.5767 - val_accuracy: 0.7500
Epoch 8/100
7/7 [======

## Problem Four: Learn Iris (multi-level classification) with Keras

In [17]:
# Load dataset
iris_df = pd.read_csv("/content/drive/MyDrive/assignment28/Iris.csv")

# Extract features and labels
X = iris_df[["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]].values
y = iris_df["Species"].replace({"Iris-versicolor": 0, "Iris-virginica": 1, "Iris-setosa": 2}).values

# Split into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
X_test, y_test = X_temp, y_temp

# Hyperparameters
learning_rate = 0.001
batch_size = 10
num_epochs = 100

# Neural network model
model = Sequential([
    Dense(50, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(100, activation='relu'),
    Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(X_val, y_val), verbose=1)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_acc)

Epoch 1/100
10/10 [==============================] - 1s 24ms/step - loss: 1.0667 - accuracy: 0.3646 - val_loss: 0.9577 - val_accuracy: 0.3750
Epoch 2/100
10/10 [==============================] - 0s 6ms/step - loss: 0.9015 - accuracy: 0.5833 - val_loss: 0.8441 - val_accuracy: 0.7083
Epoch 3/100
10/10 [==============================] - 0s 6ms/step - loss: 0.7813 - accuracy: 0.6875 - val_loss: 0.7304 - val_accuracy: 0.7083
Epoch 4/100
10/10 [==============================] - 0s 8ms/step - loss: 0.6731 - accuracy: 0.8333 - val_loss: 0.6403 - val_accuracy: 0.7083
Epoch 5/100
10/10 [==============================] - 0s 8ms/step - loss: 0.5677 - accuracy: 0.7083 - val_loss: 0.5532 - val_accuracy: 0.7083
Epoch 6/100
10/10 [==============================] - 0s 7ms/step - loss: 0.4861 - accuracy: 0.8125 - val_loss: 0.4899 - val_accuracy: 0.8333
Epoch 7/100
10/10 [==============================] - 0s 9ms/step - loss: 0.4388 - accuracy: 0.9583 - val_loss: 0.4424 - val_accuracy: 0.8333
Epoch 8/100


## Problem Five: Learning House Prices with Keras

In [18]:
# Load dataset
house_df = pd.read_csv("/content/drive/MyDrive/assignment28/train.csv")

# Extract features and labels
X = house_df[["GrLivArea", "YearBuilt"]].values
y = house_df["SalePrice"].values

# Split into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
X_test, y_test = X_temp, y_temp

# Hyperparameters
learning_rate = 0.001
batch_size = 10
num_epochs = 100

# Neural network model
model = Sequential([
    Dense(50, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(100, activation='relu'),
    Dense(1)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate), loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(X_val, y_val), verbose=1)

# Evaluate the model
test_loss = model.evaluate(X_test, y_test)
print("Test MSE:", test_loss)

Epoch 1/100
94/94 [==============================] - 1s 6ms/step - loss: 37208379392.0000 - val_loss: 30877843456.0000
Epoch 2/100
94/94 [==============================] - 0s 4ms/step - loss: 21745981440.0000 - val_loss: 7343503872.0000
Epoch 3/100
94/94 [==============================] - 1s 8ms/step - loss: 4725260800.0000 - val_loss: 3205627392.0000
Epoch 4/100
94/94 [==============================] - 0s 5ms/step - loss: 3865221888.0000 - val_loss: 3157396224.0000
Epoch 5/100
94/94 [==============================] - 1s 6ms/step - loss: 3787299840.0000 - val_loss: 3187352320.0000
Epoch 6/100
94/94 [==============================] - 0s 5ms/step - loss: 3720390144.0000 - val_loss: 2950095360.0000
Epoch 7/100
94/94 [==============================] - 0s 5ms/step - loss: 3634663424.0000 - val_loss: 2874524160.0000
Epoch 8/100
94/94 [==============================] - 0s 4ms/step - loss: 3568010240.0000 - val_loss: 2866650112.0000
Epoch 9/100
94/94 [==============================] - 1s 5ms/s

## Problem Six: Learning MNIST with Keras

In [19]:
# Load dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Reshape and normalize data
X_train = X_train.reshape(-1, 784).astype('float32') / 255
X_test = X_test.reshape(-1, 784).astype('float32') / 255

# One-hot encode labels
enc = OneHotEncoder(sparse=False)
y_train_one_hot = enc.fit_transform(y_train.reshape(-1, 1))
y_test_one_hot = enc.transform(y_test.reshape(-1, 1))

# Hyperparameters
learning_rate = 0.001
batch_size = 10
num_epochs = 10

# Neural network model
model = Sequential([
    Dense(50, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(100, activation='relu'),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train_one_hot, epochs=num_epochs, batch_size=batch_size, validation_split=0.2, verbose=1)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test_one_hot)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/10
4800/4800 [==============================] - 17s 3ms/step - loss: 0.2686 - accuracy: 0.9187 - val_loss: 0.1505 - val_accuracy: 0.9578
Epoch 2/10
4800/4800 [==============================] - 15s 3ms/step - loss: 0.1272 - accuracy: 0.9618 - val_loss: 0.1244 - val_accuracy: 0.9629
Epoch 3/10
4800/4800 [==============================] - 15s 3ms/step - loss: 0.0946 - accuracy: 0.9703 - val_loss: 0.1027 - val_accuracy: 0.9692
Epoch 4/10
4800/4800 [==============================] - 15s 3ms/step - loss: 0.0746 - accuracy: 0.9761 - val_loss: 0.1048 - val_accuracy: 0.9700
Epoch 5/10
4800/4800 [==============================] - 14s 3ms/step - loss: 0.0608 - accuracy: 0.9800 - val_loss: 0.1147 - val_accuracy: 0.9678
Epoch 6/10
4800/4800 [==============================] - 14s 3ms/step - loss: 0.0528 - accuracy: 0.9827 - val_loss: 0.1063 - val_accuracy: 0.9728
Epoch 7/10
4800/4800 [==============================] - 14s 3ms/step - loss: 0.0467 - accuracy: 0.9849 - val_loss: 0.1081 - val_ac

## Problem Seven: (Advance assignment) Rewriting to PyTorch

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Load dataset
iris_df = pd.read_csv("/content/drive/MyDrive/assignment28/Iris.csv")

# Filter only versicolor and virginica species
iris_df = iris_df[(iris_df["Species"] == "Iris-versicolor") | (iris_df["Species"] == "Iris-virginica")]

# Extract features and labels
X = iris_df[["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]].values
y = iris_df["Species"].replace({"Iris-versicolor": 0, "Iris-virginica": 1}).astype(np.int64).values

# Split into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
X_test, y_test = X_temp, y_temp

# Convert to PyTorch tensors
X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_val, y_val = torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32).view(-1, 1)
X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Hyperparameters
learning_rate = 0.001
batch_size = 10
num_epochs = 100

# DataLoader
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Neural network model
class BinaryClassifier(nn.Module):
    def __init__(self):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 50)
        self.fc2 = nn.Linear(50, 100)
        self.fc3 = nn.Linear(100, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

model = BinaryClassifier()

# Loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validate the model
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for X_batch, y_batch in val_loader:
            outputs = model(X_batch)
            val_loss += criterion(outputs, y_batch).item()
        val_loss /= len(val_loader)

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss:.4f}")

# Evaluate the model
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, y_test).item()
    test_acc = ((test_outputs >= 0.5) == y_test).float().mean().item()
    print("Test Loss:", test_loss)
    print("Test Accuracy:", test_acc)

Epoch [1/100], Loss: 0.7177, Val Loss: 0.7103
Epoch [2/100], Loss: 0.8101, Val Loss: 0.7143
Epoch [3/100], Loss: 0.6155, Val Loss: 0.6926
Epoch [4/100], Loss: 0.7101, Val Loss: 0.6873
Epoch [5/100], Loss: 0.6177, Val Loss: 0.6797
Epoch [6/100], Loss: 0.5816, Val Loss: 0.6528
Epoch [7/100], Loss: 0.5190, Val Loss: 0.6857
Epoch [8/100], Loss: 0.5599, Val Loss: 0.7012
Epoch [9/100], Loss: 0.5859, Val Loss: 0.6786
Epoch [10/100], Loss: 0.5696, Val Loss: 0.6285
Epoch [11/100], Loss: 0.5245, Val Loss: 0.6268
Epoch [12/100], Loss: 0.6203, Val Loss: 0.6389
Epoch [13/100], Loss: 0.5968, Val Loss: 0.6054
Epoch [14/100], Loss: 0.5048, Val Loss: 0.5750
Epoch [15/100], Loss: 0.4752, Val Loss: 0.5791
Epoch [16/100], Loss: 0.5210, Val Loss: 0.5319
Epoch [17/100], Loss: 0.6976, Val Loss: 0.5919
Epoch [18/100], Loss: 0.4387, Val Loss: 0.5152
Epoch [19/100], Loss: 0.4169, Val Loss: 0.4978
Epoch [20/100], Loss: 0.4613, Val Loss: 0.4951
Epoch [21/100], Loss: 0.3805, Val Loss: 0.4650
Epoch [22/100], Loss: 

In [21]:
# Load dataset
iris_df = pd.read_csv("/content/drive/MyDrive/assignment28/Iris.csv")

# Extract features and labels
X = iris_df[["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]].values
y = iris_df["Species"].replace({"Iris-versicolor": 0, "Iris-virginica": 1, "Iris-setosa": 2}).values

# Split into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
X_test, y_test = X_temp, y_temp

# Convert to PyTorch tensors
X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long)
X_val, y_val = torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.long)
X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long)

# Hyperparameters
learning_rate = 0.001
batch_size = 10
num_epochs = 100

# DataLoader
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Neural network model
class MultiClassClassifier(nn.Module):
    def __init__(self):
        super(MultiClassClassifier, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 50)
        self.fc2 = nn.Linear(50, 100)
        self.fc3 = nn.Linear(100, 3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = MultiClassClassifier()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validate the model
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for X_batch, y_batch in val_loader:
            outputs = model(X_batch)
            val_loss += criterion(outputs, y_batch).item()
        val_loss /= len(val_loader)

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss:.4f}")

# Evaluate the model
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, y_test).item()
    test_acc = (test_outputs.argmax(dim=1) == y_test).float().mean().item()
    print("Test Loss:", test_loss)
    print("Test Accuracy:", test_acc)


Epoch [1/100], Loss: 0.9794, Val Loss: 0.9435
Epoch [2/100], Loss: 0.8450, Val Loss: 0.8540
Epoch [3/100], Loss: 0.7413, Val Loss: 0.7465
Epoch [4/100], Loss: 0.6907, Val Loss: 0.6526
Epoch [5/100], Loss: 0.6312, Val Loss: 0.5606
Epoch [6/100], Loss: 0.6506, Val Loss: 0.4839
Epoch [7/100], Loss: 0.5533, Val Loss: 0.4320
Epoch [8/100], Loss: 0.4611, Val Loss: 0.4129
Epoch [9/100], Loss: 0.4924, Val Loss: 0.3685
Epoch [10/100], Loss: 0.2732, Val Loss: 0.3519
Epoch [11/100], Loss: 0.2964, Val Loss: 0.3346
Epoch [12/100], Loss: 0.2871, Val Loss: 0.3172
Epoch [13/100], Loss: 0.2046, Val Loss: 0.3007
Epoch [14/100], Loss: 0.1727, Val Loss: 0.2800
Epoch [15/100], Loss: 0.2478, Val Loss: 0.2649
Epoch [16/100], Loss: 0.2523, Val Loss: 0.2719
Epoch [17/100], Loss: 0.1846, Val Loss: 0.2577
Epoch [18/100], Loss: 0.1275, Val Loss: 0.2407
Epoch [19/100], Loss: 0.1348, Val Loss: 0.2409
Epoch [20/100], Loss: 0.1827, Val Loss: 0.2265
Epoch [21/100], Loss: 0.1359, Val Loss: 0.2227
Epoch [22/100], Loss: 

In [22]:
# Load dataset
house_df = pd.read_csv("/content/drive/MyDrive/assignment28/train.csv")

# Extract features and labels
X = house_df[["GrLivArea", "YearBuilt"]].values
y = house_df["SalePrice"].values

# Split into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
X_test, y_test = X_temp, y_temp

# Convert to PyTorch tensors
X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_val, y_val = torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32).view(-1, 1)
X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Hyperparameters
learning_rate = 0.001
batch_size = 10
num_epochs = 100

# DataLoader
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Neural network model
class HousePriceRegressor(nn.Module):
    def __init__(self):
        super(HousePriceRegressor, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 50)
        self.fc2 = nn.Linear(50, 100)
        self.fc3 = nn.Linear(100, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = HousePriceRegressor()

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validate the model
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for X_batch, y_batch in val_loader:
            outputs = model(X_batch)
            val_loss += criterion(outputs, y_batch).item()
        val_loss /= len(val_loader)

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss:.4f}")

# Evaluate the model
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, y_test).item()
    print("Test MSE:", test_loss)


Epoch [1/100], Loss: 21776162816.0000, Val Loss: 30235884202.6667
Epoch [2/100], Loss: 4147607040.0000, Val Loss: 6708678672.0000
Epoch [3/100], Loss: 2273930752.0000, Val Loss: 3290898984.0000
Epoch [4/100], Loss: 1119881344.0000, Val Loss: 3191094834.6667
Epoch [5/100], Loss: 874884864.0000, Val Loss: 2987294256.0000
Epoch [6/100], Loss: 2969547008.0000, Val Loss: 2875024309.3333
Epoch [7/100], Loss: 12514043904.0000, Val Loss: 2759914698.6667
Epoch [8/100], Loss: 1422142976.0000, Val Loss: 2744521570.6667
Epoch [9/100], Loss: 16364515328.0000, Val Loss: 2640905077.3333
Epoch [10/100], Loss: 710188800.0000, Val Loss: 2651531728.0000
Epoch [11/100], Loss: 5516817920.0000, Val Loss: 2516788186.6667
Epoch [12/100], Loss: 9341265920.0000, Val Loss: 2459893485.3333
Epoch [13/100], Loss: 1620108672.0000, Val Loss: 2430738194.6667
Epoch [14/100], Loss: 826764288.0000, Val Loss: 2455953173.3333
Epoch [15/100], Loss: 684907968.0000, Val Loss: 2409616138.6667
Epoch [16/100], Loss: 703214016.00

In [24]:
# Load dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Reshape and normalize data
X_train = X_train.reshape(-1, 784).astype('float32') / 255
X_test = X_test.reshape(-1, 784).astype('float32') / 255

# One-hot encode labels
enc = OneHotEncoder(sparse=False)
y_train_one_hot = enc.fit_transform(y_train.reshape(-1, 1))
y_test_one_hot = enc.transform(y_test.reshape(-1, 1))

# Convert to PyTorch tensors
X_train, y_train_one_hot = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train_one_hot, dtype=torch.float32)
X_test, y_test_one_hot = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test_one_hot, dtype=torch.float32)

# Hyperparameters
learning_rate = 0.001
batch_size = 10
num_epochs = 10

# DataLoader
train_dataset = TensorDataset(X_train, y_train_one_hot)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Neural network model
class MNISTClassifier(nn.Module):
    def __init__(self):
        super(MNISTClassifier, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 50)
        self.fc2 = nn.Linear(50, 100)
        self.fc3 = nn.Linear(100, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.softmax(self.fc3(x), dim=1)
        return x

model = MNISTClassifier()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Evaluate the model
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, y_test_one_hot).item()
    test_acc = (test_outputs.argmax(dim=1) == y_test_one_hot.argmax(dim=1)).float().mean().item()
    print("Test Loss:", test_loss)
    print("Test Accuracy:", test_acc)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch [1/10], Loss: 1.4614
Epoch [2/10], Loss: 1.5186
Epoch [3/10], Loss: 1.4612
Epoch [4/10], Loss: 1.4680
Epoch [5/10], Loss: 1.4612
Epoch [6/10], Loss: 1.4612
Epoch [7/10], Loss: 1.4631
Epoch [8/10], Loss: 1.4612
Epoch [9/10], Loss: 1.4612
Epoch [10/10], Loss: 1.4612
Test Loss: 1.4968708753585815
Test Accuracy: 0.9643999934196472


## Problem Eight: (Advance assignment) Comparison of frameworks
When comparing TensorFlow and Keras, there are several aspects to consider:

Here is a summary of the differences between Tensorflow, Keras, and PyTorch:

Calculation Speed: All three frameworks have comparable calculation speeds, with each excelling in specific scenarios based on model architecture and hardware utilization.

Number of Lines of Code and Readability: Keras stands out for its conciseness and readability due to its high-level API, making it beginner-friendly. PyTorch offers a balance between readability and flexibility, while TensorFlow, being more low-level, might require more code for similar tasks.

Functions Provided: TensorFlow boasts a comprehensive suite of functions, catering to both research and production needs. PyTorch provides a dynamic computation graph, allowing for greater flexibility during model development. Keras, with its streamlined API, focuses on ease of use for common deep learning tasks.

In summary, Keras is the go-to choice for beginners and rapid prototyping due to its simplicity and ease of use. PyTorch, with its balance of flexibility and readability, is favored by researchers for its dynamic computation graphs. TensorFlow, with its extensive functionalities, caters to both research and production, offering fine-grained control and scalability for experienced users.